In [1]:
# Importação das bibliotecas a serem utilizadas durante todo o processo

import pandas as pd
import numpy as np
from datetime import datetime
import time
import math
import random
import sys
import pytz
import os

In [2]:
# Definicção de qual loja e período irá rodar o script

loja = 1
periodo = 2

In [3]:
# Leitura do arquivo da tabela 2

start = time.time()
df = pd.read_csv(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\loja_{loja}_tabela_2_1_p{periodo}_v1.csv', encoding='utf-8')
df['cod novo'] = df['cod novo'].astype(str)
df

C:\Users\vinic\AppData\Local\Temp\ipykernel_10208\3957445940.py:4: DtypeWarning: Columns (22,28) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\loja_{loja}_tabela_2_1_p{periodo}_v1.csv', encoding='utf-8')


,Unnamed: 0,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,tipo,COD_ITEM,IND_OPER,SUB_TIPO,...,DESCRICAO,CODIGO_BARRA,UNIDADE,N C M,CEST,CNPJ EMITENTE,CNPJ DESTINATARIO,Valor Base Cálculo ICMS Substituição Tributária,vBCST,FONTE
0,0,35191003129126000159550020000027291157936550,2019-10-01,1411,1,94771,grupo,94771,0,-1,...,Nimesulida Novaq 100mg 12S,SEM GTIN,UN,30049079.0,1300200.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
1,1,35191003129126000159550020000027301158021920,2019-10-02,1411,1,148098,grupo,148098,0,-1,...,Kit Concare Manicure,SEM GTIN,UN,82142000.0,2802600.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
2,2,35191003129126000159550020000027311158028019,2019-10-02,1411,1,155283,grupo,155283,0,-1,...,Creme Prev Assadur 45g Le Blue,SEM GTIN,UN,33049910.0,2800800.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
3,3,35191003129126000159550020000027311158028019,2019-10-02,1411,2,155283,grupo,155283,0,-1,...,Creme Prev Assadur 45g Le Blue,SEM GTIN,UN,33049910.0,2800800.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
4,4,35191003129126000159550020000027321158122292,2019-10-03,1411,1,78550,grupo,78550,0,-1,...,Ibuprofeno Prati 600mg 20S,SEM GTIN,UN,30049029.0,1300200.0,3.129126e+12,0.000000e+00,NaN,0.0,entrada_oprprp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493477,493477,35200503129126000159550020000031271171064100,2020-05-09,5411,1,211409,grupo,211409,1,-1,...,DULCOGAS 125MG CAP GEL 10 S,7891058022495.0,UN,30049099.0,1300301.0,3.129126e+12,2.784996e+13,NaN,0.0,saida_nfe
493478,493478,35200703129126000159550020000032161174195310,2020-07-04,5411,1,224790,grupo,224790,1,-1,...,AGUA MICELAR LE BLUE 150ML,7898623355309.0,UN,33049990.0,2001500.0,3.129126e+12,2.784996e+13,NaN,0.0,saida_nfe
493479,493479,35200703129126000159550020000032171174195334,2020-07-04,5411,1,224790,grupo,224790,1,-1,...,AGUA MICELAR LE BLUE 150ML,7898623355309.0,UN,33049990.0,2001500.0,3.129126e+12,2.784996e+13,NaN,0.0,saida_nfe
493480,493480,35201003129126000159550020000033711179926296,2020-10-08,5411,1,44831,NaN,44831,1,-1,...,ALDIJET AMP 1ML,7899420500732.0,UN,30066000.0,1300500.0,3.129126e+12,5.651966e+12,NaN,0.0,saida_nfe


In [4]:
def calcular_ressarcimento(df):
    '''
    Função para cálculo do valor de ressarcimento
    para a loja e de outras informações para o
    arquivo TXT final
    
    '''
    
    df = df.drop('Unnamed: 0', axis=1)
    df = df.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                               ascending=[True, True, True, True])
    
    # Criação de dataset para cálculo do ICMS inicial

    data = df[['cod novo', 'DATA', 'QTD_CAT', 'IND_OPER']]
    
    # Transformação da coluna 'DATA' para o tipo correto
    data['DATA'] = pd.to_datetime(data['DATA'], format='%Y-%m-%d')
    
    # Ordenação do dataset para cálculo correto do ICMS inicial

    data['IND_OPER'] = data['IND_OPER'].astype(int)
    data = data.sort_values(by=['cod novo','DATA', 'IND_OPER'])
    
    # Cáculo dos saldos das operações para definição da Quantidade Inicial

    data['OPER'] = np.where(data['IND_OPER'] == 0, data['QTD_CAT'], -data['QTD_CAT'])
    data['SALDO'] = data.groupby('cod novo')['OPER'].cumsum()
    
    # Criação da coluna para as informações da Quantidade Inicial

    min_saldo = data.groupby('cod novo')['SALDO'].transform('min')

    # Set 'QTD_INI' to the minimum 'SALDO' value if it's less than or equal to 0,
    # otherwise set it to 0
    data['QTD_INI'] = np.where(min_saldo <= 0, min_saldo.abs(), 0)

    # Get the first rows within each group
    first_rows = data.groupby('cod novo').head(1)

    # Set 'QTD_INI' to NaN for all rows except the last rows within each group
    data['QTD_INI'] = np.where(data.index.isin(first_rows.index), data['QTD_INI'], np.nan)

    # Update 'QTD_INI' with calculated values for the last rows
    data.loc[first_rows.index, 'QTD_INI'] = first_rows['QTD_INI']
    
    # Cálculo do ICMS inicial 

    data['SUB_TIPO'] = df['SUB_TIPO']
    data['ICMS_TOT_0'] = df['ICMS_TOT']
    data['VALOR_UNIT'] = np.where((data['SUB_TIPO'] == 1) & (data['IND_OPER'] == 0),
                                  data['ICMS_TOT_0'] / data['QTD_CAT'],
                                  np.nan)

    grouped_data = data.groupby('cod novo').agg({
        'VALOR_UNIT': 'mean',
        'QTD_INI': 'first'
    })

    # Vectorized calculation for icms_init
    grouped_data['ICMS_INI'] = grouped_data['VALOR_UNIT'] * (1 - 0.3 * np.random.random(size=len(grouped_data))) * grouped_data['QTD_INI']
    data = data.merge(grouped_data[['ICMS_INI']], how='left', left_on='cod novo', right_index=True)

    mask = data.duplicated(subset='cod novo', keep='first')

    # Assign 'ICMS_INI' values to NaN where mask is True, indicating it's not the first row for the 'COD_ITEM'
    data['ICMS_INI'] = np.where(mask, np.nan, data['ICMS_INI'])
    
    tabela_2 = df[['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod novo', 'IND_OPER', 'SUB_TIPO', 'QTD_CAT', 'ALIQUOTA', 'FONTE']]
    
    tabela_2['DATA'] = pd.to_datetime(tabela_2['DATA'], format='%Y-%m-%d')
    tabela_2 = tabela_2.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                               ascending=[True, True, True, True])
    
    tabela_2['QTD_INI'] = data['QTD_INI']
    tabela_2['ICMS_INI'] = data['ICMS_INI']
    tabela_2['ICMS_INI'] = np.where((tabela_2['QTD_INI'] == 0) | (tabela_2['ICMS_INI'].isnull() & tabela_2['QTD_INI'].notnull()),
                                    0,
                                    tabela_2['ICMS_INI'])
    tabela_2 = tabela_2[['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod novo', 'IND_OPER', 
                        'SUB_TIPO', 'QTD_CAT','QTD_INI','ICMS_INI', 'ALIQUOTA', 'FONTE']]
    
    mask_condition1 = (tabela_2['IND_OPER'] == 0) & (tabela_2['SUB_TIPO'] == 1)
    mask_condition2 = (tabela_2['IND_OPER'] != 0) & (tabela_2['SUB_TIPO'] == -1)

    qtd_ent1_devolv_ent = list(np.where(mask_condition1, tabela_2['QTD_CAT'], 0) + np.where(mask_condition2, -tabela_2['QTD_CAT'], 0))
    tabela_2['QTD_ent1_devolv_ent'] = qtd_ent1_devolv_ent
    
    tabela_2['ICMS_TOT'] = df['ICMS_TOT']
    tabela_2['ICMS_TOT'] = tabela_2['ICMS_TOT'].fillna(0)
    
    mask = tabela_2['QTD_ent1_devolv_ent'] > 0

    tabela_2['ICMS_TOT_ent_unit'] = np.where(mask, tabela_2['ICMS_TOT'] / tabela_2['QTD_ent1_devolv_ent'], 0)
    
    tabela_2['ULT_ICMS_TOT_ent_unit'] = np.nan

    # Calculate the ULT_ICMS_TOT_ent_unit using vectorized operations
    values = np.where(
        (tabela_2['QTD_INI'] != 0) & 
        (tabela_2['cod novo'] == tabela_2['cod novo'].shift()) & 
        (tabela_2['QTD_ent1_devolv_ent'].shift() > 0),
        tabela_2['ICMS_TOT_ent_unit'].shift(),
        np.where(
            (tabela_2['QTD_INI'] == 0),
            0.01,
            tabela_2['ICMS_INI'] / tabela_2['QTD_INI']
        )
    )
    
    tabela_2['tipo'] = df['tipo']
    
    # Assign values to the 'ULT_ICMS_TOT_ent_unit' column
    tabela_2['ULT_ICMS_TOT_ent_unit'] = values

    # Forward fill NaN values in 'ULT_ICMS_TOT_ent_unit' column
    tabela_2['ULT_ICMS_TOT_ent_unit'] = tabela_2['ULT_ICMS_TOT_ent_unit'].ffill()
    
    mask_condition1 = (tabela_2['IND_OPER'] == 1) & (tabela_2['SUB_TIPO'] == 1)
    mask_condition2 = (tabela_2['IND_OPER'] != 1) & (tabela_2['SUB_TIPO'] == -1)

    qtd_saida_1_devolv_saida_t = np.where(mask_condition1, tabela_2['QTD_CAT'], 0) + np.where(mask_condition2, -tabela_2['QTD_CAT'], 0)

    tabela_2['qtd_saida_1_devolv_saida'] = qtd_saida_1_devolv_saida_t
    
    tabela_2['QTD_SALDO'] = tabela_2.groupby('cod novo').apply(lambda x: (
    x['QTD_INI'].iloc[0] + (x['QTD_ent1_devolv_ent'] - x['qtd_saida_1_devolv_saida']).cumsum())).reset_index(level=0, drop=True)
    
    tabela_2['REF'] = tabela_2['DATA'].astype(str)
    tabela_2['REF'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%m-%Y') for x in tabela_2['REF']]

    # Definição do primeiro valor de ICMS_SAIDA_UNI
    icms_saida_uni = [] 

    # Definição do primeiro valor de ULT_ICMS_SAIDA_UNI
    ult_icms_saida_uni = [] 

    # Definição do primeiro valor de ICMS_SAIDA
    icms_saida = [] 

    # Definição do primeiro valor de ICMS_TOT_SALDO
    icms_tot_saldo = [] 

    # Definição do primeiro valor de ICMS_TOT_1
    icms_tot_1 = []

    # Definição de todos os outros valores para cada campo acima
    for code in tabela_2['cod novo'].unique():

        icms_saida_uni_int = []
        ult_icms_saida_uni_int = []
        icms_saida_int = []
        icms_tot_saldo_int = []
        icms_tot_1_int = []
        qtd_saida_1_devolv_saida = list(tabela_2[tabela_2['cod novo'] == code]['qtd_saida_1_devolv_saida'])
        qtd_saldo = list(tabela_2[tabela_2['cod novo'] == code]['QTD_SALDO'])
        qtd_cat = list(tabela_2[tabela_2['cod novo'] == code]['QTD_CAT'])
        ref = list(tabela_2[tabela_2['cod novo'] == code]['REF'])

        for i,row in tabela_2[tabela_2['cod novo']==code].reset_index(drop=True).iterrows():

        # ICMS_SAIDA_UNI

            if i == 0:
                if qtd_saida_1_devolv_saida[0] > 0:
                    if row['QTD_INI'] == 0:
                        icms_saida_uni_int.append(0)
                    else:  
                        icms_saida_uni_int.append(row['ICMS_INI'] / row['QTD_INI'])
                else:
                    icms_saida_uni_int.append(0)

            else:
                if (qtd_saida_1_devolv_saida[i] != 0) and (qtd_saldo[i-1] != 0):  
                    icms_saida_uni_int.append(icms_tot_saldo_int[i-1] / qtd_saldo[i-1])               
                else:      
                    icms_saida_uni_int.append(0)

        # ULT_ICMS_SAIDA_UNI
            if i== 0:
                if row['QTD_INI'] == 0:
                    ult_icms_saida_uni_int.append(0.01)
                else:           
                    ult_icms_saida_uni_int.append(
                        row['ICMS_INI'] / row['QTD_INI'] if row['QTD_INI'] != 0 else float("nan"))
            else:
                if qtd_saida_1_devolv_saida[i-1] > 0:
                    ult_icms_saida_uni_int.append(icms_saida_uni_int[i-1])
                else:
                    ult_icms_saida_uni_int.append(ult_icms_saida_uni_int[i-1])


        # ICMS_SAIDA
            if qtd_saida_1_devolv_saida[i] < 0:
                icms_saida_int.append(qtd_saida_1_devolv_saida[i]*ult_icms_saida_uni_int[i])
            else:
                icms_saida_int.append(icms_saida_uni_int[i]*qtd_saida_1_devolv_saida[i])

        # ICMS_TOT_1
            if i == 0:
                if (row['QTD_ent1_devolv_ent'] < 0) & (row['QTD_CAT'] == row['QTD_INI']) :
                    icms_tot_1_int.append(-row['ICMS_INI'])
                else:
                    icms_tot_1_int.append(row['ICMS_TOT'])
            else:
                if row['QTD_ent1_devolv_ent'] < 0:
                    if (qtd_cat[i] == qtd_saldo[i-1]):
                        icms_tot_1_int.append(-1*icms_tot_saldo_int[i-1])
                    else:
                        icms_tot_1_int.append(max(row['QTD_ent1_devolv_ent'] * row['ULT_ICMS_TOT_ent_unit'], -1*icms_tot_saldo_int[i-1]))
                else:
                    icms_tot_1_int.append(row['ICMS_TOT']) 

        # ICMS_TOT_SALDO
            if i == 0:
                icms_tot_saldo_int.append(row['ICMS_INI'] + icms_tot_1_int[i] - icms_saida_int[i])
            else:
                icms_tot_saldo_int.append(icms_tot_saldo_int[i-1] + icms_tot_1_int[i] - icms_saida_int[i])


        icms_saida_uni += icms_saida_uni_int
        ult_icms_saida_uni += ult_icms_saida_uni_int
        icms_saida += icms_saida_int
        icms_tot_saldo += icms_tot_saldo_int
        icms_tot_1 += icms_tot_1_int

    tabela_2['ICMS_SAIDA_UNI'] = icms_saida_uni
    tabela_2['ULT_ICMS_SAIDA_UNI'] = ult_icms_saida_uni
    tabela_2['ICMS_SAIDA'] = icms_saida
    tabela_2['ICMS_TOT_SALDO'] = icms_tot_saldo
    tabela_2['ICMS_TOT_SALDO'] = np.where((tabela_2['ICMS_TOT_SALDO'] < 0.01) & (tabela_2['ICMS_TOT_SALDO'] > -0.01),
                                         0,
                                         tabela_2['ICMS_TOT_SALDO'])
    tabela_2['ICMS_TOT_1'] = icms_tot_1
    
    tabela_2['ICMS_TOT_PCAT'] = np.where((tabela_2['qtd_saida_1_devolv_saida'] < 0),
                                     np.abs(tabela_2['ICMS_SAIDA_UNI']),
                                     np.where((tabela_2['QTD_ent1_devolv_ent'] != 0),
                                              np.abs(tabela_2['ICMS_TOT_1']),
                                              np.nan))
    
    if tabela_2[tabela_2['ICMS_TOT_PCAT'] < 0].shape[0] > 0:
        mensagem('EXISTE ICMS TOT PCAT NEGATIVO. FAVOR CHECAR!!')
        print(mensagem)
    
    tabela_2['VLR_CONF_0'] = df['VL_CONFR_0']
    
    mask = tabela_2['qtd_saida_1_devolv_saida'] > 0
    vlr_confr_unit = np.where(mask, tabela_2['VLR_CONF_0']/tabela_2['qtd_saida_1_devolv_saida'], np.nan)

    tabela_2['VLR_CONFR_UNIT'] = vlr_confr_unit
    
    ult_vlr_confr_unit = [0]
    for i in range(1, tabela_2.shape[0]):
        if tabela_2.iloc[i]['cod novo'] != tabela_2.iloc[i-1]['cod novo']:
            ult_vlr_confr_unit.append(0)
        else:
            if math.isnan(tabela_2.iloc[i-1]['VLR_CONFR_UNIT']):
                ult_vlr_confr_unit.append(ult_vlr_confr_unit[i-1])
            else:
                ult_vlr_confr_unit.append(tabela_2.iloc[i-1]['VLR_CONFR_UNIT'])

    tabela_2['ULT_VLR_CONFR_UNIT'] = ult_vlr_confr_unit
    
    tabela_2['VLR_CONFR_1'] = np.where(tabela_2['qtd_saida_1_devolv_saida'] > 0,
                                   tabela_2['VLR_CONF_0'],
                                   np.where(tabela_2['qtd_saida_1_devolv_saida'] < 0,
                                            tabela_2['ULT_VLR_CONFR_UNIT'] * tabela_2['qtd_saida_1_devolv_saida'],
                                            0))
    
    tabela_2['VLR_CONFR_1'] = np.where(tabela_2['VLR_CONFR_1'] <= 0.01,
                                              0.01,
                                              tabela_2['VLR_CONFR_1'])
    
    tabela_2['COD_LEGAL'] = df['COD_LEGAL']
    
    mask = np.abs(tabela_2['ICMS_SAIDA']) > np.abs(tabela_2['VLR_CONFR_1'])
    vlr_ressarc = np.where(tabela_2['COD_LEGAL'] == 1, (np.where(mask, tabela_2['ICMS_SAIDA']-tabela_2['VLR_CONFR_1'], 0)),0)

    tabela_2['VLR_RESSARCIMENTO'] = vlr_ressarc
    
    mask = np.abs(tabela_2['ICMS_SAIDA']) < np.abs(tabela_2['VLR_CONFR_1'])
    vlr_compl = np.where(tabela_2['COD_LEGAL'] == 1, np.where(mask, -tabela_2['ICMS_SAIDA']+tabela_2['VLR_CONFR_1'], 0),0)

    tabela_2['VLR_COMPLEMENTO'] = vlr_compl
    
    tabela_2['COD_LEGAL_PCAT'] = np.where((tabela_2['COD_LEGAL'] == 1) & (tabela_2['VLR_RESSARCIMENTO'] != 0) & (tabela_2['tipo'] != 'outros2') & (tabela_2['ALIQUOTA'] != 0), 1, 0)
    tabela_2.loc[pd.isna(tabela_2['COD_LEGAL']), 'COD_LEGAL_PCAT'] = np.nan
    
    tabela_2['VLR_CONFR_PCAT'] = np.where(tabela_2['COD_LEGAL_PCAT'] == 1, np.abs(tabela_2['VLR_CONFR_1']), np.nan)
    if tabela_2[(tabela_2['COD_LEGAL'] == 1) & (tabela_2['VLR_CONFR_PCAT'] <= 0)].shape[0] > 0:
        mensagem = 'Há valores de confronto inconsistentes. Favor checar.'
        print(mensagem)
        with open(os.path.join(diretorio_principal, f'Error_loja_{loja}_p{periodo}'), 'w') as arquivo:
            arquivo.write(mensagem)
        sys.exit()
    
    mask = (tabela_2['cod novo'].shift(-1) != tabela_2['cod novo']) | (tabela_2['REF'].shift(-1) != tabela_2['REF'])
    saldo_final_mes_qtd = np.where(mask, tabela_2['QTD_SALDO'], np.nan)

    tabela_2['SALDO_FINAL_MES_QTD'] = saldo_final_mes_qtd
    
    mask = (tabela_2['cod novo'].shift(-1) != tabela_2['cod novo']) | (tabela_2['REF'].shift(-1) != tabela_2['REF'])
    saldo_final_mes_icms = np.where(mask, tabela_2['ICMS_TOT_SALDO'], np.nan)

    tabela_2['SALDO_FINAL_MES_ICMS'] = saldo_final_mes_icms
    
    tabela_2['VALOR'] = df['VALOR']
    tabela_2['Valor Base Cálculo ICMS ST Retido Operação Anterior'] = df['Valor Base Cálculo ICMS ST Retido Operação Anterior']
    tabela_2['Valor Complementar'] = df['Valor Complementar']
    tabela_2['vBCST'] = df['vBCST']
    tabela_2['Valor ICMS Substituição Tributária'] = df['Valor ICMS Substituição Tributária']
    tabela_2['Valor ICMS Operação'] = df['Valor ICMS Operação']
    tabela_2['CNPJ EMITENTE'] = df['CNPJ EMITENTE']
    tabela_2['COD_ITEM'] = df['COD_ITEM']


    tabela_2.rename(columns={'ICMS_TOT': 'ICMS_TOT_0',}, inplace=True)
    tabela_2['ICMS_TOT'] = df['ICMS_TOT']
    
    tabela_2 = tabela_2[['CHV_DOC', 'DATA', 'CFOP', 'NUM_ITEM', 'cod novo', 'IND_OPER', 'SUB_TIPO', 'QTD_CAT',
                        'QTD_INI', 'ICMS_INI', 'QTD_ent1_devolv_ent', 'ICMS_TOT_0', 'ICMS_TOT_ent_unit',
                        'ULT_ICMS_TOT_ent_unit', 'ICMS_TOT_1', 'qtd_saida_1_devolv_saida', 'ICMS_SAIDA_UNI',
                        'ULT_ICMS_SAIDA_UNI', 'ICMS_SAIDA', 'ICMS_TOT_PCAT', 'VLR_CONF_0', 'VLR_CONFR_UNIT',
                        'ULT_VLR_CONFR_UNIT', 'VLR_CONFR_1', 'QTD_SALDO', 'ICMS_TOT_SALDO', 'VLR_RESSARCIMENTO',
                        'VLR_COMPLEMENTO', 'COD_LEGAL', 'COD_LEGAL_PCAT', 'VLR_CONFR_PCAT', 'SALDO_FINAL_MES_QTD',
                        'SALDO_FINAL_MES_ICMS', 'ALIQUOTA', 'VALOR', 'Valor Base Cálculo ICMS ST Retido Operação Anterior',
                        'Valor Complementar', 'Valor ICMS Substituição Tributária', 'Valor ICMS Operação', 
                        'ICMS_TOT', 'COD_ITEM', 'CNPJ EMITENTE', 'tipo', 'vBCST', 
                        'FONTE']]
    
    return tabela_2

In [5]:
tabela_2 = calcular_ressarcimento(df)

C:\Users\vinic\AppData\Local\Temp\ipykernel_10208\2000691713.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DATA'] = pd.to_datetime(data['DATA'], format='%Y-%m-%d')
C:\Users\vinic\AppData\Local\Temp\ipykernel_10208\2000691713.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['IND_OPER'] = data['IND_OPER'].astype(int)
C:\Users\vinic\AppData\Local\Temp\ipykernel_10208\2000691713.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

In [6]:
tabela_2

,CHV_DOC,DATA,CFOP,NUM_ITEM,cod novo,IND_OPER,SUB_TIPO,QTD_CAT,QTD_INI,ICMS_INI,...,Valor Base Cálculo ICMS ST Retido Operação Anterior,Valor Complementar,Valor ICMS Substituição Tributária,Valor ICMS Operação,ICMS_TOT,COD_ITEM,CNPJ EMITENTE,tipo,vBCST,FONTE
209619,35191003129126000159590006031131098680305954,2019-10-24,5405,3,100073,1,1,1,1.0,12.541354,...,0.00,NaN,0.0,0.0,NaN,100073,NaN,grupo,0.000000,dfe
9117,35191027849963000110550010005964131005945510,2019-10-29,1403,20,100073,0,1,1,NaN,NaN,...,63.62,11.96,0.0,0.0,13.6044,100073,2.784996e+13,grupo,75.582468,entrada_sp
14104,35191127849963000110550010006084971006065827,2019-11-09,1403,7,100073,0,1,1,NaN,NaN,...,63.62,11.96,0.0,0.0,13.6044,100073,2.784996e+13,grupo,75.582468,entrada_sp
209620,35191203129126000159590006031131212496179820,2019-12-05,5405,1,100073,1,1,1,NaN,NaN,...,0.00,NaN,0.0,0.0,NaN,100073,NaN,grupo,0.000000,dfe
24064,35191227849963000110550010006364621006344734,2019-12-08,1403,36,100073,0,1,1,NaN,NaN,...,63.72,11.86,0.0,0.0,13.6044,100073,2.784996e+13,grupo,75.582468,entrada_sp
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
162453,35201203129126000159590007086090676940106578,2020-12-23,5405,3,99234,1,1,1,NaN,NaN,...,0.00,NaN,0.0,0.0,NaN,99234,NaN,grupo,0.000000,dfe
162450,35201203129126000159590005812821041405820715,2020-12-24,5405,5,99234,1,1,1,NaN,NaN,...,0.00,NaN,0.0,0.0,NaN,99234,NaN,grupo,0.000000,dfe
141676,35201227849963000110550010010262221010226748,2020-12-29,1403,19,99234,0,1,1,NaN,NaN,...,1.88,3.75,0.0,0.0,1.4075,99234,2.784996e+13,grupo,5.632534,entrada_sp
241965,35201203129126000159590007086090696680045137,2020-12-31,5405,4,99234,1,1,1,NaN,NaN,...,0.00,NaN,0.0,0.0,NaN,99234,NaN,grupo,0.000000,dfe


In [7]:
tabela_2['VLR_RESSARCIMENTO'].sum() / tabela_2[tabela_2['CFOP'] == 5405]['VALOR'].sum()

0.04427852529652348

In [8]:
meta_por_loja = pd.read_excel(os.path.join('C:\\Bots\\Bot_transmissao\\Geracao CAT\\Referencias', 'GTI_12.xlsx'))
try:
    meta = meta_por_loja[(meta_por_loja['Loja'] == loja) & (meta_por_loja['periodo'] == f'p{periodo}')]['ESTIMATI3'].values[0]
except:
    meta = 0.085
            
def gti_pra_cima(tabela_2, meta):
    
    '''
    Função para aplicação de GTI para lojas
    com ressarcimento abaixo da porcentagem estabelecida
    como meta
    
    '''

    valor = tabela_2[tabela_2['CFOP'] == 5405]['VALOR'].sum()
    ressarc = tabela_2['VLR_RESSARCIMENTO'].sum()
    fator = 0.1
    
    # 1ª Iteração
    
    tabela_2['cod novo'] = tabela_2['cod novo'].astype(str)
    pivot_ressarc = tabela_2[tabela_2['tipo'] == 'grupo'].pivot_table(index='cod novo', values='VLR_RESSARCIMENTO', aggfunc='sum')
    pivot_ressarc = pivot_ressarc[pivot_ressarc['VLR_RESSARCIMENTO'] > 0]
    pivot_ressarc = pivot_ressarc.sort_values(by='VLR_RESSARCIMENTO', ascending=False)  
    
    incr_max_ref = pivot_ressarc.iloc[54].values[0]
    incr_max = pivot_ressarc.iloc[55:]
    incr_max['Diferenças'] = incr_max_ref - incr_max['VLR_RESSARCIMENTO']
    incr_max = incr_max.sort_values(by='cod novo')
    
    qtd_saida = tabela_2[tabela_2['CFOP'] == 5405].pivot_table(index='cod novo', values='QTD_CAT', aggfunc='sum')
    incr_max_unit = incr_max.merge(qtd_saida, on='cod novo', how='left')
    incr_max_unit['Incremento maximo unitario'] = incr_max_unit['Diferenças'] / incr_max_unit['QTD_CAT']
    incr_max_unit = incr_max_unit.reset_index()
    
    tabela_2 = tabela_2.merge(incr_max_unit[['cod novo', 'Incremento maximo unitario']], on='cod novo', how='left')
    tabela_2['Incremento maximo unitario'] = np.where(tabela_2['ICMS_TOT'].fillna(0) == 0,
                                                 0,
                                                 tabela_2['Incremento maximo unitario'])
    
    tabela_2['Incremento maximo unitario'] = tabela_2['Incremento maximo unitario'].fillna(0)
    
    tabela_2['ICMS_TOT_FINAL'] = np.where(tabela_2['ICMS_TOT'].fillna(0) != 0,
                                     tabela_2['ICMS_TOT'] + (tabela_2['Incremento maximo unitario']*fator)*tabela_2['QTD_CAT'],
                                     0)
    
    tabela_2['bc ST op ant'] = np.where(tabela_2['ALIQUOTA'] != 0,
                                   tabela_2['ICMS_TOT_FINAL'] / (tabela_2['ALIQUOTA']/100),
                                   tabela_2['Valor Base Cálculo ICMS ST Retido Operação Anterior'])
    
    tabela_2['bc ST op ant comple'] = tabela_2['bc ST op ant'] - tabela_2['Valor Base Cálculo ICMS ST Retido Operação Anterior']
    
    df_final = df.copy()
    df_final = df_final.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                                   ascending=[True, True, True, True]).reset_index().drop(['index'], axis=1)
    
    df_final['bc ST op ant comple'] = np.where(tabela_2['tipo'] == 'grupo',
                                               tabela_2['bc ST op ant comple'],
                                               tabela_2['Valor Complementar'])
    
    df_final.rename(columns={'ICMS_TOT': 'ICMS_TOT_ORIG',
                            'Valor Complementar': 'Complementar Original',
                            'bc ST op ant comple': 'Valor Complementar'}, inplace=True)
    
    df_final['ICMS_TOT'] = np.where(df_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'] <= df_final['vBCST'],
             np.maximum(df_final['Valor ICMS Operação'].fillna(0) + df_final['Valor ICMS Substituição Tributária'].fillna(0), (df_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'] + df_final['Valor Complementar'].fillna(0)) * df_final['ALIQUOTA'].fillna(0).astype(float)/100),
             np.maximum(df_final['Valor ICMS Operação'].fillna(0) + df_final['Valor ICMS Substituição Tributária'].fillna(0), (df_final['vBCST'] + df_final['Valor Complementar'].fillna(0)) * df_final['ALIQUOTA'].fillna(0).astype(float)/100))

    
    tabela_2_final = calcular_ressarcimento(df_final)
    
    # 2ª Iteração
    ressarc_tot_0_1 = tabela_2_final['VLR_RESSARCIMENTO'].sum()
    delta_fator_0_1 = ressarc_tot_0_1 - ressarc
    meta = meta
    meta_ressarc = valor * meta
    meta_delta = meta_ressarc - ressarc
    meta_fator_0_1 = meta_delta/delta_fator_0_1
    meta_fator = meta_fator_0_1*0.1*0.95
    
    tabela_2_final['cod novo'] = tabela_2_final['cod novo'].astype(str)
    pivot_ressarc_final = tabela_2_final[tabela_2_final['tipo'] == 'grupo'].pivot_table(index='cod novo', values='VLR_RESSARCIMENTO', aggfunc='sum')
    pivot_ressarc_final = pivot_ressarc_final[pivot_ressarc_final['VLR_RESSARCIMENTO'] > 0]
    pivot_ressarc_final = pivot_ressarc_final.sort_values(by='VLR_RESSARCIMENTO', ascending=False)
    
    
    incr_max_ref_final = pivot_ressarc_final.iloc[54].values[0]
    incr_max_final = pivot_ressarc_final.iloc[55:]
    incr_max_final['Diferenças'] = incr_max_ref_final - incr_max_final['VLR_RESSARCIMENTO']
    incr_max_final = incr_max_final.sort_values(by='cod novo')
    
    qtd_saida_final = tabela_2_final[tabela_2_final['CFOP'] == 5405].pivot_table(index='cod novo', values='QTD_CAT', aggfunc='sum')
    incr_max_unit_final = incr_max_final.merge(qtd_saida_final, on='cod novo', how='left')
    incr_max_unit_final['Incremento maximo unitario'] = incr_max_unit_final['Diferenças'] / incr_max_unit_final['QTD_CAT']
    
    incr_max_unit_final = incr_max_unit_final.reset_index()
    
    tabela_2_final['ICMS_TOT_ORIG'] = df_final['ICMS_TOT_ORIG']
    tabela_2_final = tabela_2_final.merge(incr_max_unit_final[['cod novo', 'Incremento maximo unitario']], on='cod novo', how='left')
    tabela_2_final['Incremento maximo unitario'] = np.where(tabela_2_final['ICMS_TOT_ORIG'].fillna(0) == 0,
                                                 0,
                                                 tabela_2_final['Incremento maximo unitario'])
    tabela_2_final['Incremento maximo unitario'] = tabela_2_final['Incremento maximo unitario'].fillna(0)
    
    tabela_2_final['ICMS_TOT_FINAL'] = np.where(tabela_2_final['ICMS_TOT_ORIG'].fillna(0) != 0,
                                     tabela_2_final['ICMS_TOT_ORIG'] + (tabela_2_final['Incremento maximo unitario']*meta_fator)*tabela_2_final['QTD_CAT'],
                                     0)
    
    tabela_2_final['bc ST op ant'] = np.where(tabela_2_final['ALIQUOTA'] != 0,
                                   tabela_2_final['ICMS_TOT_FINAL'] / (tabela_2_final['ALIQUOTA']/100),
                                   tabela_2_final['Valor Base Cálculo ICMS ST Retido Operação Anterior'])
    
    tabela_2_final['bc ST op ant comple'] = tabela_2_final['bc ST op ant'] - tabela_2_final['Valor Base Cálculo ICMS ST Retido Operação Anterior']
    
    
    df_fim = df.copy()
    df_fim = df_fim.sort_values(by=['cod novo', 'DATA', 'IND_OPER', 'SUB_TIPO'], 
                                   ascending=[True, True, True, True]).reset_index().drop(['index'], axis=1)
    
    df_fim['bc ST op ant comple'] = np.where(tabela_2_final['tipo'] == 'grupo',
                                               tabela_2_final['bc ST op ant comple'],
                                               tabela_2_final['Valor Complementar'])
    
    df_fim.rename(columns={'ICMS_TOT': 'ICMS_TOT_ORIG',
                            'Valor Complementar': 'Complementar Original',
                            'bc ST op ant comple': 'Valor Complementar'}, inplace=True)
    
    df_fim['ICMS_TOT'] = np.where(df_fim['Valor Base Cálculo ICMS ST Retido Operação Anterior'] <= df_fim['vBCST'],
             np.maximum(df_fim['Valor ICMS Operação'].fillna(0) + df_fim['Valor ICMS Substituição Tributária'].fillna(0), (df_fim['Valor Base Cálculo ICMS ST Retido Operação Anterior'] + df_fim['Valor Complementar'].fillna(0)) * df_fim['ALIQUOTA'].fillna(0).astype(float)/100),
             np.maximum(df_fim['Valor ICMS Operação'].fillna(0) + df_fim['Valor ICMS Substituição Tributária'].fillna(0), (df_fim['vBCST'] + df_fim['Valor Complementar'].fillna(0)) * df_fim['ALIQUOTA'].fillna(0).astype(float)/100))
    
    tabela_2_fim = calcular_ressarcimento(df_fim)
    
    return tabela_2_fim

def gti_pra_baixo(tabela_2, meta):
    '''
    Função para aplicação de GTI para lojas
    com ressarcimento acima da porcentagem estabelecida
    como meta
    
    '''
    
    ressarc = tabela_2['VLR_RESSARCIMENTO'].sum()
    meta_ressarc = tabela_2[tabela_2['CFOP'] == 5405]['VALOR'].sum() * meta
    diferenca = ressarc-meta_ressarc
    pivot_ressarc = tabela_2.pivot_table(index='cod novo', values='VLR_RESSARCIMENTO', aggfunc='sum')
    pivot_ressarc = pivot_ressarc[pivot_ressarc['VLR_RESSARCIMENTO'] > 0].sort_values(by='VLR_RESSARCIMENTO',
                                                                                     ascending=True)
    soma = 0
    i = 0
    cods = []
    while i < len(pivot_ressarc) and (soma + pivot_ressarc['VLR_RESSARCIMENTO'].iloc[i]) <= diferenca:
        soma += pivot_ressarc['VLR_RESSARCIMENTO'].iloc[i]
        cods.append(pivot_ressarc.index[i])
        i += 1
    
    df['COD_LEGAL'] = np.where(df['cod novo'].isin(cods),
                              np.where(df['COD_LEGAL'].isnull(), df['COD_LEGAL'], 0),
                              df['COD_LEGAL'])
    
    df['VL_CONFR_0'] = np.where(df['cod novo'].isin(cods),
                               np.nan,
                               df['VL_CONFR_0'])
    
    tabela_2_final = calcular_ressarcimento(df)
    
    return tabela_2_final

In [9]:
if (tabela_2['VLR_RESSARCIMENTO'].sum() / tabela_2[tabela_2['CFOP'] == 5405]['VALOR'].sum()) < meta:
    tabela_2_def = gti_pra_cima(tabela_2, meta)
#     with open(os.path.join('C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}', f'Log_GTI_{loja}_p{periodo}'), 'w') as arquivo:
#         arquivo.write('LOJA COM GTI PARA CIMA')
            
elif (tabela_2['VLR_RESSARCIMENTO'].sum() / tabela_2[tabela_2['CFOP'] == 5405]['VALOR'].sum()) > meta:
#     with open(os.path.join('C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}', f'Log_GTI_{loja}_p{periodo}'), 'w') as arquivo:
#         arquivo.write('LOJA COM GTI PARA BAIXO')
    print('ATENÇÃO. LOJA COM PORCENTAGEM DE RESSARCIMENTO ACIMA DA MARGEM')
    print('+'*50)
    print('\n')
    print('REPETINDO. ATENÇÃO. LOJA COM PORCENTAGEM DE RESSARCIMENTO ACIMA DA MARGEM')
    tabela_2_def = gti_pra_baixo(tabela_2, meta)

C:\Users\vinic\AppData\Local\Temp\ipykernel_10208\2738891347.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  incr_max['Diferenças'] = incr_max_ref - incr_max['VLR_RESSARCIMENTO']
C:\Users\vinic\AppData\Local\Temp\ipykernel_10208\2000691713.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['DATA'] = pd.to_datetime(data['DATA'], format='%Y-%m-%d')
C:\Users\vinic\AppData\Local\Temp\ipykernel_10208\2000691713.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

In [10]:
tabela_2_def['VLR_RESSARCIMENTO'].sum() / tabela_2_def[tabela_2_def['CFOP'] == 5405]['VALOR'].sum()

0.08127309938956044

In [11]:
# Criação das funções para geração dos arquivos Excel

def gerar_excel_lojas_pequenas(tabela_2):
    '''
    Função para exportação da Ficha 3 em arquivo excel
    para o caso de o arquivo de ficha 3 da loja tenha
    menos de 1 milhão de linhas
    
    '''
    
    path = f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_v1.xlsx'
    with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
        tabela_2.to_excel(writer, sheet_name='ficha3', index=False)
        
def gerar_excel_lojas_grandes(tabela_2):
    '''
    Função para exportação da Ficha 3 em arquivo excel
    para o caso de o arquivo de ficha 3 da loja tenha
    mais de 1 milhão de linhas
    
    '''
    
    cods = list(tabela_2['cod novo'].unique())
    l = len(cods)

    fat_div = 2

    while True:

        cods_p1 = cods[:int(l/fat_div)]
        cods_p2 = cods[int(l/fat_div):]

        tabela_2_1 = tabela_2[tabela_2['cod novo'].isin(cods_p1)]
        tabela_2_2 = tabela_2[tabela_2['cod novo'].isin(cods_p2)]

        if tabela_2_1.shape[0] > 1000000:
            fat_div += 1

        else:
            break
    
    # Geração do arquivo final referente ao dataframe 1
    path = f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_parte1_v1.xlsx'
    with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
        tabela_2_1.to_excel(writer, sheet_name='ficha3', index=False)
        
    # Geração do arquivo final referente ao dataframe 2
    path = f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\ficha_3_loja_{loja}_p{periodo}_parte2_v1.xlsx'
    with pd.ExcelWriter(path, engine='xlsxwriter') as writer:
        tabela_2_2.to_excel(writer, sheet_name='ficha3', index=False)

In [12]:
if tabela_2.shape[0] > 1000000:
    print('+'*50)
    print('ATENÇÃO, LOJA GRANDE. ATENÇÃO, LOJA GRANDE')
    print('+'*50)
    
    gerar_excel_lojas_grandes(tabela_2_def)
    
else:
    gerar_excel_lojas_pequenas(tabela_2_def)

In [13]:
tabela_2_def['REF'] = [datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m') for x in tabela_2_def['DATA'].astype(str)]
pivot = tabela_2_def.pivot_table(index='REF', values=['VLR_RESSARCIMENTO', 'VLR_COMPLEMENTO'], aggfunc='sum').sort_values(by='REF')
pivot_fat = tabela_2_def[tabela_2_def['CFOP'] == 5405].pivot_table(index='REF', values='VALOR', aggfunc='sum').sort_values(by='REF')
fuso_horario_brasilia = pytz.timezone('America/Sao_Paulo')
data_geracao_brasilia = datetime.now(fuso_horario_brasilia).strftime('%Y-%m-%d %H:%M:%S')

pivot['VALOR'] = pivot_fat['VALOR']
pivot['Data'] = data_geracao_brasilia
cnpj = tabela_2_def[tabela_2_def['FONTE'] == 'dfe']['CHV_DOC'].str.slice(6,20).unique()
pivot['CNPJ'] = list(cnpj)*len(pivot)

pivot

,VLR_COMPLEMENTO,VLR_RESSARCIMENTO,VALOR,Data,CNPJ
REF,,,,,
2019-10,9230.090117,17677.241305,234641.62,2024-02-28 17:41:37,03129126000159
2019-11,9212.106706,20318.730147,237611.47,2024-02-28 17:41:37,03129126000159
2019-12,11462.443252,24525.429181,300875.03,2024-02-28 17:41:37,03129126000159
2020-01,12094.439101,26224.592812,322002.67,2024-02-28 17:41:37,03129126000159
2020-02,8883.442604,20864.995076,243830.59,2024-02-28 17:41:37,03129126000159
2020-03,9928.760149,24099.254649,274375.99,2024-02-28 17:41:37,03129126000159
2020-04,7816.793614,22429.278518,237437.15,2024-02-28 17:41:37,03129126000159
2020-05,8688.637205,18820.611922,231115.28,2024-02-28 17:41:37,03129126000159
2020-06,7354.646377,17605.169081,208860.49,2024-02-28 17:41:37,03129126000159


In [14]:
pivot['VALOR'].sum()

3893690.44

In [15]:
pivot.rename(columns={'REF': 'DATA'})
pivot.to_excel(f'C:\\Bots\\Bot_transmissao\\Geracao CAT\\Loja_{loja}\\relatorio_mensal_ressarcimento_complemento_{loja}_p{periodo}_v1.xlsx')

In [16]:
end = time.time()
print(end - start)

4903.111194849014


In [17]:
tabela_2_def[tabela_2_def['CFOP'].isin([1403, 5411])]['COD_LEGAL_PCAT'].unique()

array([nan])